In [3]:
pip install git+https://github.com/boilingdata/py-boilingdata@v0.2.9 pandas numpy

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/boilingdata/py-boilingdata (to revision v0.2.8) to /private/var/folders/vy/p7dz__m15lb1pbm6xk1b9gxw0000gn/T/pip-req-build-0vj36aff
  Running command git clone --filter=blob:none --quiet https://github.com/boilingdata/py-boilingdata /private/var/folders/vy/p7dz__m15lb1pbm6xk1b9gxw0000gn/T/pip-req-build-0vj36aff
  Running command git checkout -q 441de5147572b5cd5374b02f78066f922d0d91d7
  Resolved https://github.com/boilingdata/py-boilingdata to commit 441de5147572b5cd5374b02f78066f922d0d91d7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
from dotenv import load_dotenv
from py_boilingdata import BoilingData

load_dotenv()  # take environment variables from .env

boiling = BoilingData(log_level="INFO")
await boiling.connect()

async def bd(sql):
    resp = await boiling.execute(sql)
    try:
        return pd.DataFrame(resp)
    except:
        return resp

ValueError: Missing username (BD_USERNAME) and/or password (BD_PASSWORD) environment variable(s)

To get a list of Data Sets shared to you, you can call:

In [2]:
resp = await bd("SELECT shareName, fromEmail, lifeTime, schedule FROM boilingshares LIMIT 100;")
print(resp)

                shareName             fromEmail lifeTime     schedule
0          taxi_locations  demo@boilingdata.com       1h  * * * * * *
1  taxi_locations_limited    dforsber@gmail.com       1h  * * * * * *
2          taxi_locations    dforsber@gmail.com       1h  * * * * * *
3               demo_full    dforsber@gmail.com       1h  * * * * * *


In [3]:
await bd("SELECT * FROM share('dforsber@gmail.com:taxi_locations') LIMIT 2")

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone


In [5]:
await bd("SELECT * FROM parquet_scan('s3://boilingdata-demo/test.parquet') LIMIT 5")

,registration_dttm,id,first_name,last_name,email,gender,ip_address,cc,country,birthdate,salary,title,comments
0,2016-02-03 07:55:29+00,1,Amanda,Jordan,ajordan0@com.com,Female,1.197.201.2,6759521864920116,Indonesia,3/8/1971,49756.53,Internal Auditor,1E+02
1,2016-02-03 17:04:03+00,2,Albert,Freeman,afreeman1@is.gd,Male,218.111.175.34,,Canada,1/16/1968,150280.17,Accountant IV,
2,2016-02-03 01:09:31+00,3,Evelyn,Morgan,emorgan2@altervista.org,Female,7.161.136.94,6767119071901597,Russia,2/1/1960,144972.51,Structural Engineer,
3,2016-02-03 00:36:21+00,4,Denise,Riley,driley3@gmpg.org,Female,140.35.109.83,3576031598965625,China,4/8/1997,90263.05,Senior Cost Accountant,
4,2016-02-03 05:05:31+00,5,Carlos,Burns,cburns4@miitbeian.gov.cn,,169.113.235.40,5602256255204850,South Africa,,NaN,,


In [ ]:
await boiling.close()